### downloading census data for comparison with synth pop outputs

In [1]:
%load_ext autoreload
%autoreload 2
from synthpop.census_helpers import Census
from synthpop import categorizer as cat
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_columns', 500)

This notebook is being used to process ACS data obtained from the ACS API. 

First some notes about the structure of how synthpop works. 
It has a few dependencies: 
- **census**, a python library that is a wrapper for the U.S. Census Bureau API
- **pandas** a python library used to create and work with dataframes (like tables)
- **numpy**
- **os** which allows you to set up a development environment (not actually entirely necessary)



**Step 1:**  
Set API key. (If you don't already have you can get one [here](https://api.census.gov/data/key_signup.html))

In [2]:
# Dare's Census API key
c = Census("d95e144b39e17f929287714b0b8ba9768cecdc9f")

**Step 2:**  
Set up state and county variables to use for the state/county you are working with

In [3]:
# set state to North Carolina
stateFips = "37"
# set county to Mecklenburg
countyFips = "119"

**Step 3:**  
Define household variables of interest

In [4]:
income_columns = ['B19001_0%02dE'%i for i in range(1, 18)]
income_columns_moe = ['B19001_0%02dM'%i for i in range(1, 18)]
vehicle_columns = ['B08201_0%02dE'%i for i in range(1, 7)]
vehicle_columns_moe = ['B08201_0%02dM'%i for i in range(1, 7)]

workers_columns = ['B08202_0%02dE'%i for i in range(1, 6)]
families_columns = ['B11001_001E', 'B11001_002E']
families_columns_moe = ['B11001_001M', 'B11001_002M']
# traveltimesex_columns = ['B08013_001E','B08013_002E','B08013_003E']
census_col = families_columns + families_columns_moe+income_columns + income_columns_moe + vehicle_columns + vehicle_columns_moe
h_acs = c.tract_query(census_col, stateFips, countyFips)


In [6]:
##Note: it is also possible to do this at the block and tract level. See example below. 
## We are not using this
income_columns = ['B19001_0%02dE'%i for i in range(1, 18)]
vehicle_columns = ['B08201_0%02dE'%i for i in range(1, 7)]
workers_columns = ['B08202_0%02dE'%i for i in range(1, 6)]
families_columns = ['B11001_001E', 'B11001_002E']
# traveltimesex_columns = ['B08013_001E','B08013_002E','B08013_003E']
block_group_columns = income_columns + families_columns
tract_columns = vehicle_columns + workers_columns 
# + traveltimesex_columns
#function for getting block and tract level data. merges to the block level
h_acs_block = c.block_group_and_tract_query(block_group_columns,
                tract_columns, stateFips, countyFips, 
                merge_columns=['tract', 'county', 'state'],
                block_group_size_attr="B11001_001E",
                tract_size_attr="B08201_001E")

**Step 4:**  
Define person-level variables of interest

In [7]:
population = ['B01001_001E']
# margin of error = _moe in all columns
population_moe = ['B01001_001M']
sex = ['B01001_002E', 'B01001_026E']
sex_moe = ['B01001_002M', 'B01001_026M']
race = ['B02001_0%02dE'%i for i in range(1,11)]
race_moe = ['B02001_0%02dM'%i for i in range(1,11)]                                     
male_age_columns = ['B01001_0%02dE'%i for i in range(3,26)]
male_age_columns_moe = ['B01001_0%02dM'%i for i in range(3,26)]
female_age_columns = ['B01001_0%02dE'%i for i in range(27,50)]
female_age_columns_moe = ['B01001_0%02dM'%i for i in range(27,50)]
hispanic = ['B03003_002E', 'B03003_003E']
hispanic_moe = ['B03003_002M', 'B03003_003M']

                                             
all_columns = population + sex + race + male_age_columns + hispanic+ hispanic_moe+ female_age_columns+population_moe+sex_moe+race_moe+male_age_columns_moe+female_age_columns_moe
p_acs_tract = c.tract_query(all_columns, stateFips, countyFips)


In [24]:
##calculate standard error for person level columns
margin_error = [population_moe, sex_moe, race_moe, male_age_columns_moe,female_age_columns_moe]


0.21888412017167383

0.21888412017167383

**Step 5:**  
Categorize the ACS household data into categories we can read

This step includes calculating aggregate errors for the ACS variables we are using. 

    This was calculated using 
    SE1 + SE2 + ...+ SEN

    Where SE = Standard Error = Margin of Error / Z
    And Z is defined by ACS as 1.645

    See [this](https://www2.census.gov/programs-surveys/acs/tech_docs/statistical_testing/2015StatisticalTesting5year.pdf) guide from ACS for further info. 

In [20]:
h_acs_cat = cat.categorize(h_acs, {
    ("households", "total"): "B11001_001E",
    ("income", "lt35"): "B19001_002E + B19001_003E + B19001_004E + "
                        "B19001_005E + B19001_006E + B19001_007E",
    ("income", "lt35_me"): "(B19001_002M + B19001_003M + B19001_004M +"
                        "B19001_005M + B19001_006M + B19001_007M)/1.645",
    ("income", "gt35-lt100"): "B19001_008E + B19001_009E + "
                        "B19001_010E + B19001_011E + B19001_012E"
                        "+ B19001_013E",
    ("income", "gt35-lt100_me"): "(B19001_008M + B19001_009M + "
                        "B19001_010M + B19001_011M + B19001_012M"
                        "+ B19001_013M)/1.645",
    ("income", "gt100"): "B19001_014E + B19001_015E + B19001_016E"
                        "+ B19001_017E",
    ("income", "gt100_me"): "(B19001_014M + B19001_015M + B19001_016M"
                        "+ B19001_017M)/1.645",
    ("cars", "none"): "B08201_002E",
    ("cars", "none_me"): "B08201_002M",
    ("cars", "one"): "B08201_003E",
    ("cars", "one_me"): "B08201_003M",
    ("cars", "two or more"): "B08201_004E + B08201_005E + B08201_006E",
    ("cars", "two or more_me"): "(B08201_004M + B08201_005M + B08201_006M)/1.645",
#     ("workers", "none"): "B08202_002E",
#     ("workers", "one"): "B08202_003E",
#     ("workers", "two or more"): "B08202_004E + B08202_005E",
#     ("traveltime","total"):"B08013_001E",
#     ("traveltime","male"):"B08013_002E",
#     ("traveltime","female"):"B08013_003E"
}, index_cols=['tract'])

h_acs_cat.columns = h_acs_cat.columns.droplevel()

In [21]:
h_acs_cat.head()

cat_value,none,none_me,one,one_me,two or more,two or more_me,total,gt100,gt100_me,gt35-lt100,gt35-lt100_me,lt35,lt35_me
tract,,,,,,,,,,,,,
000100,426,180,1327,229,1072,181.762918,2825,1476,329.483283,867,291.793313,482,263.221884
000300,155,40,206,37,120,31.610942,481,152,55.319149,93,48.632219,236,75.987842
000400,278,103,1014,161,472,97.872340,1764,650,187.841945,794,230.395137,320,150.759878
000500,441,156,1271,214,783,144.680851,2495,944,353.799392,921,274.772036,630,231.003040
000600,235,79,832,118,502,98.480243,1569,379,135.562310,644,199.392097,546,193.920973


**Step 8:**  
Categorize the ACS person data into categories we can read


This step includes calculating aggregate errors for the ACS variables we are using. 

    This was calculated using 
    SE1 + SE2 + ...+ SEN

    Where SE = Standard Error = Margin of Error / Z
    And Z is defined by ACS as 1.645

    See [this](https://www2.census.gov/programs-surveys/acs/tech_docs/statistical_testing/2015StatisticalTesting5year.pdf) guide from ACS for further info. 

In [10]:
p_acs_cat = cat.categorize(p_acs_tract, {
    ("population", "total"): "B01001_001E",
    ("population", "total_me"): "B01001_001M",
    ("age", "19 and under"): "B01001_003E + B01001_004E + B01001_005E + "
                             "B01001_006E + B01001_007E + B01001_027E + "
                             "B01001_028E + B01001_029E + B01001_030E + "
                             "B01001_031E",
    ("age", "19 and under_me"): "(B01001_003M + B01001_004M + B01001_005M + "
                             "B01001_006M + B01001_007M + B01001_027M + "
                             "B01001_028M + B01001_029M + B01001_030M + "
                             "B01001_031M)/1.645",
    ("age", "20 to 35"): "B01001_008E + B01001_009E + B01001_010E + "
                         "B01001_011E + B01001_012E + B01001_032E + "
                         "B01001_033E + B01001_034E + B01001_035E + "
                         "B01001_036E",
    ("age", "20 to 35_me"): "(B01001_008M + B01001_009M + B01001_010M + "
                         "B01001_011M + B01001_012M + B01001_032M + "
                         "B01001_033M + B01001_034M + B01001_035M + "
                         "B01001_036M)/1.645",
    ("age", "35 to 60"): "B01001_013E + B01001_014E + B01001_015E + "
                         "B01001_016E + B01001_017E + B01001_037E + "
                         "B01001_038E + B01001_039E + B01001_040E + "
                         "B01001_041E",
    ("age", "35 to 60_me"): "(B01001_013M + B01001_014M + B01001_015M + "
                         "B01001_016M + B01001_017M + B01001_037M + "
                         "B01001_038M + B01001_039M + B01001_040M + "
                         "B01001_041M)/1.645",
    ("age", "above 60"): "B01001_018E + B01001_019E + B01001_020E + "
                         "B01001_021E + B01001_022E + B01001_023E + "
                         "B01001_024E + B01001_025E + B01001_042E + "
                         "B01001_043E + B01001_044E + B01001_045E + "
                         "B01001_046E + B01001_047E + B01001_048E + "
                         "B01001_049E", 
    ("age", "above 60_me"): "(B01001_018M + B01001_019M + B01001_020M + "
                         "B01001_021M + B01001_022M + B01001_023M + "
                         "B01001_024M + B01001_025M + B01001_042M + "
                         "B01001_043M + B01001_044M + B01001_045M + "
                         "B01001_046M + B01001_047M + B01001_048M + "
                         "B01001_049M)/1.645", 
    ("race", "white"):   "B02001_002E",
    ("race", "white_me"):   "B02001_002M",
    ("race", "black"):   "B02001_003E",
    ("race", "black_me"):   "B02001_003M",
    ("race", "asian"):   "B02001_005E",
    ("race", "asian_me"):   "B02001_005M",
    ("race", "other"):   "B02001_004E + B02001_006E + B02001_007E + "
                         "B02001_008E",
    ("race", "other_me"):   "(B02001_004M + B02001_006M + B02001_007M + "
                         "B02001_008M)/1.645",
    ("sex", "male"):     "B01001_002E",
    ("sex", "male_me"):     "B01001_002M",
    ("sex", "female"):   "B01001_026E",
    ("sex", "female_me"):   "B01001_026M",
    ("hispanic", "yes"): "B03003_003E",
    ("hispanic", "yes_me"): "B03003_003M",
    ("hispanic", "no"): "B03003_002E",
    ("hispanic", "no_me"): "B03003_002M",
}, index_cols=['tract'])
p_acs_cat

cat_name           age                                                \
cat_value 19 and under 19 and under_me 20 to 35 20 to 35_me 35 to 60   
tract                                                                  
000100             244      158.054711     2713  733.130699     1544   
000300              18       56.534954      204   93.617021      340   
000400              52       84.498480     1751  417.021277      661   
000500             834      412.158055     2791  708.814590      981   
000600             409      202.431611     1588  416.413374      671   
000700             120       88.753799      262  117.325228      344   
000800            1008      281.458967      822  225.531915      849   
000900             322      167.781155      516  209.118541      638   
001000             463      209.726444      901  281.458967      942   
001100             437      181.155015      685  234.042553      983   
001200            1088      488.753799     1172  471.732523     2245   
001300             661      289.969605     1013  352.583587     1359   
001400             368      190.273556      945  291.185410     1068   
001504            2841      975.075988     1418  699.088146     2265   
001505            1046      382.978723      754  328.875380     1229   
001507            1297      483.890578     1133  449.848024     1222   
001508            1773      606.079027     1306  469.908815     2170   
001509             837      330.091185      698  320.972644     1022   
001510             692      352.583587      484  278.419453     1110   
001603             939      403.647416     1105  372.036474     1204   
001605             417      204.863222      588  242.553191      689   
001606            1353      511.854103      881  362.917933     1629   
001607             781      438.297872      796  328.267477      939   
001608            1078      384.802432     1009  344.072948     1344   
001609             830      275.987842      918  307.598784     1046   
001701            1091      421.884498     1653  434.042553     1173   
001702            1460      530.699088     1786  522.188450     1608   
001801             256      128.875380      441  163.525836      554   
001802             491      250.455927     1051  288.145897     1037   
001910            1894      485.714286     1692  468.085106     1664   
...                ...             ...      ...         ...      ...   
006009             728      271.732523      699  277.203647     1051   
006010            2555      940.425532     1228  503.951368     2174   
006103            1051      365.957447      608  281.458967     1660   
006104            2670      835.258359     2355  778.115502     3442   
006105             930      297.264438      560  232.826748     1201   
006106             822      301.519757     1170  398.176292     1620   
006107            1282      549.544073     1747  585.410334     1918   
006108            1447      477.811550     1135  393.920973     1658   
006109             962      433.434650     1571  479.027356     1472   
006203             929      396.960486      738  375.683891     2304   
006204            1613      507.598784      510  345.288754     3176   
006208             809      349.544073      907  318.541033      947   
006209             863      398.784195     1076  426.747720     1902   
006210            1254      337.386018      705  249.240122     1700   
006211            2168      572.036474      679  341.033435     2676   
006212             866      316.109422      972  338.601824     1265   
006213            1912      578.115502     1132  387.234043     2379   
006214            1173      374.468085      724  297.264438     1614   
006215            1197      398.176292     1056  350.759878     1757   
006302            2188      691.793313     1431  481.458967     2831   
006303            1703      485.106383     1091  426.747720     1959   
006304            1988      608.510638  

In [15]:
# dropping top level of column index
p_acs_cat.columns = p_acs_cat.columns.droplevel()

In [18]:
# renaming hispanic columns
p_acs_cat= p_acs_cat.rename(columns={'no_me':'nonhispanic_me',
                          'no':'nonhispanic','yes_me':'hispanic_me',
                         'yes':'hispanic'})

In [22]:
p_acs_cat.to_csv('data_outputs/20190330_census_aggregates/37119_people_meck.csv')
h_acs_cat.to_csv('data_outputs/20190330_census_aggregates/37119_households_meck.csv')